# AC209a Milestone 4: Baseline Recommendation Models

**Andrew Ross, Sophie Hilgard, Reiko Nishihara, Nick Hoernle**

In this notebook, we'll try various baseline models for recommendation and compare their performance.

We'll mainly use root mean squared error as our performance metric, in part because it's a good metric (interpretable, has units of stars), and in part because that's what most other recommendation systems, papers we found, and even software packages use to measure performance, so we can more easily compare our models to state of the art benchmarks.

First let's import packages and load our data:

In [1]:
import sys; sys.path.append('../util')
from load_yelp_data import load_yelp_dataframe, restaurants_and_bars_in, train_test_split_reviews
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import graphlab as gl
import sklearn.metrics
%matplotlib inline

from matrix_factorization_recommender import MatrixFactorizationRecommender
from simple_averaging_recommender import SimpleAveragingRecommender

In [2]:
businesses = load_yelp_dataframe('businesses')
reviews = load_yelp_dataframe('reviews')
users = load_yelp_dataframe('users')

We thought it would be interesting if we tried limiting our data to just restaurants and bars in Phoenix (and for comparison, Las Vegas) to support some fine-grained location-based analysis that we're still thinking about. So let's just look at Phoenix for the remainder of this notebook, maybe jumping back to compare Phoenix RMSEs to global RMSEs for comparison:

In [3]:
phoenix_restaurants, phoenix_reviews, phoenix_users = restaurants_and_bars_in('Phoenix', businesses, reviews, users)

In [17]:
phoenix_restaurants.head(2)

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,location
2377,"{'Accepts Credit Cards': True, 'Noise Level': ...",2377,"['Sandwiches', 'Pizza', 'Chicken Wings', 'Rest...",Phoenix,"2819 N Central Ave\nPhoenix, AZ 85004","{'Monday': {'close': '00:00', 'open': '10:00'}...",33.479482,-112.073681,Domino's Pizza,[],True,20,3.0,AZ,business,Phoenix
2378,"{'Accepts Credit Cards': True, 'Noise Level': ...",2378,"['American (New)', 'Sandwiches', 'Restaurants']",Phoenix,"1850 N Central Ave\nPhoenix, AZ 85004",{},33.468547,-112.075085,Viad Tower Restaurants,[],True,6,3.5,AZ,business,Phoenix


In [18]:
def utility_matrix_fullness(reviews):
    return len(reviews) / float(len(reviews.business_id.unique()) * len(reviews.user_id.unique()))

In [23]:
print 'Overall utility matrix fullness: {:.4%}'.format(utility_matrix_fullness(reviews))
print 'Phoenix utility matrix fullness: {:.4%}'.format(utility_matrix_fullness(phoenix_reviews))

Overall utility matrix fullness: 0.0046%
Phoenix utility matrix fullness: 0.0374%


So limiting to just restaurants and bars in Phoenix makes our utility matrix less sparse by about an order of magnitude. That makes sense intuitively, because we expect the network of restaurants and reviewers to be more highly connected when all the reviewers and restaurants are in the same physical location, and in addition to allowing for more interesting side-analyses, it should hopefully also improve the accuracy of our models.

Let's split our Phoenix restaurant/bar data into a training and test set, and evaluate the performance of various models.

In [32]:
def X_and_y_of(df):
    return df[['user_id', 'business_id']].values, df['stars'].values

reviews_train, reviews_test = train_test_split_reviews(phoenix_reviews)

X_train, y_train = X_and_y_of(reviews_train)
X_test, y_test = X_and_y_of(reviews_test)

In [31]:
def summarize_performance(model):
    name = model.__class__.__name__
    print '{} train RMSE: {}'.format(name, model.rmse(X_train, y_train))
    print '{} test RMSE: {}'.format(name, model.rmse(X_test, y_test))
    print '{} train R^2: {}'.format(name, model.r2_score(X_train, y_train))
    print '{} test R^2: {}'.format(name, model.r2_score(X_test, y_test))

### Simple Averaging 
The first baseline model we want to try is simple averaging (whose code you can see [here](https://github.com/NickHoernle/Data-Science-Project-AM209a/blob/master/util/simple_averaging_recommender.py)). All that simple averaging does is compute the average rating across the entire dataset as well as average deviations from the global mean for each user and each business. Then for any new user, business pair, it predicts that just the global mean plus the user and business-specific baselines. If we haven't seen a particular user or business in our training set, we predict its baseline to equal 0.

In [25]:
simple_avg = SimpleAveragingRecommender()
simple_avg.fit(X_train, y_train)
summarize_performance(simple_avg)

SimpleAveragingRecommender train RMSE: 0.976644727132
SimpleAveragingRecommender test RMSE: 1.34699149129
SimpleAveragingRecommender train R^2: 0.477786155479
SimpleAveragingRecommender test R^2: 0.0102154548537


So these results make sense; we do a fair amount better on the training set than the test set,
but our test RMSE isn't that bad. Our test R^2 is close to 0 though slightly above it, which makes
sense for an average.

In [this citation](http://cs229.stanford.edu/proj2014/Chee%20Hoon%20Ha,%20Yelp%20Recommendation%20System%20Using%20Advanced%20Collaborative%20Filtering.pdf)
(which is another exploration of recommendation algorithms applied to the Yelp dataset),
they report an RMSE of 1.47 for simple averaging. If we try applying our
simple averaging to the global dataset (with shuffle-split cross-validation):

In [33]:
global_rmses = []
local_rmses = []

for _ in range(10):
    reviews_train_global, reviews_test_global = train_test_split_reviews(reviews, seed=None)
    reviews_train_local, reviews_test_local = train_test_split_reviews(phoenix_reviews, seed=None)
    simple_avg.fit(*X_and_y_of(reviews_train_global))
    global_rmses.append(simple_avg.rmse(*X_and_y_of(reviews_test_global)))
    simple_avg.fit(*X_and_y_of(reviews_train_local))
    local_rmses.append(simple_avg.rmse(*X_and_y_of(reviews_test_local)))
    
print 'Local RMSE {}, global RMSE {}'.format(np.mean(local_rmses), np.mean(global_rmses))

Local RMSE 1.34745239586, global RMSE 1.36724780709


Strangely, although our global RMSE is worse than our local RMSE (which we'd expect), we still do better than most of the citations we found just with simple averaging.

### Matrix Factorization

The second baseline model we tried was based on [Koren, Bell, and Volinsky's work](https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf) on latent factor models realized using matrix factorization techniques. In this framing of the problem, we assume that the full utility matrix of all users' ratings for all businesses, which we don't actually have (and has $|B|\cdot|U|$ entries, where $|B|$ is the number of businesses and $|U|$ the number of users), is nevertheless explainable using a much smaller number of latent factors, which are essentially user and business vectors of dimension $f$ whose inner product generates the predicted rating. So the idea is that if we can find skinny $|B| \times f$ and $f \times |U|$ matrices that do a good job generating the ratings we do have in our very incomplete utility matrix, perhaps with some additional regularizing assumptions, then we can approximate those latent factors and make good predictions about ratings we're still missing.

In practice, to find such matrices, we need to use optimization techniques on each of the individual elements on the matrices, and the algorithms get complicated. However, there are at least a few existing software packages which implement those algorithms, including [Vowpal Wabbit](https://github.com/JohnLangford/vowpal_wabbit) (VW). We found an [example](https://github.com/JohnLangford/vowpal_wabbit/wiki/Matrix-factorization-example) of how to use VW to run matrix factorizations, and so following that, we wrote a very minimal [Python wrapper](https://github.com/NickHoernle/Data-Science-Project-AM209a/blob/master/util/matrix_factorization_recommender.py) around just that matrix factorization routine. As in the example, we let the number of latent factors per user/business $f$ be 10, we use L2 regularization, and we set the same learning rate parameters for the optimization process. It takes a few minutes to train, but the results definitely beat simple averaging:

In [26]:
matrix_fac = MatrixFactorizationRecommender()
matrix_fac.fit(X_train, y_train)
summarize_performance(matrix_fac)

Model file already exists, skipping vowpal-wabbit fitting
MatrixFactorizationRecommender train RMSE: 1.19438634692
MatrixFactorizationRecommender test RMSE: 1.19719273998
MatrixFactorizationRecommender train R^2: 0.218975203111
MatrixFactorizationRecommender test R^2: 0.218121785367


What's especially nice about this is that our test RMSE/R^2 is almost as good as our train RMSE/R^2, despite them being totally separate. This may indicate that the matrix factorization method is more robust to certain kinds of overfitting than other recommendation algorithms.

### Other Methods

We also looked at several recommendation algorithms included in a library called [GraphLab Create](https://turi.com/products/create/docs/graphlab.toolkits.recommender.html), which is kind of a higher-powered though slightly more proprietary competitor to scikit-learn. We haven't explored all of the recommendation algorithms in this library fully yet, but they include techniques akin to simple averaging, matrix factorization, and collaborative filtering. To use it, first we need to load our data into a graphlab "SFrame" (GraphLab's scalable, sometimes S3-backed version of a data frame):

In [34]:
train_sframe = gl.SFrame(reviews_train[['business_id', 'user_id', 'stars']])
test_sframe = gl.SFrame(reviews_test[['business_id', 'user_id', 'stars']])

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1480273879.log


This non-commercial license of GraphLab Create for academic use is assigned to andrew_ross@g.harvard.edu and will expire on November 23, 2017.


We can then use it to train recommendation models:

In [35]:
pop_model = gl.popularity_recommender.create(
    train_sframe, user_id='user_id', item_id='business_id', target='stars', verbose=False)
sim_model = gl.item_similarity_recommender.create(
    train_sframe, user_id='user_id', item_id='business_id', target='stars', verbose=False)
fac_model = gl.factorization_recommender.create(
    train_sframe, user_id='user_id', item_id='business_id', target='stars', verbose=False)
rank_fac_model = gl.ranking_factorization_recommender.create(
    train_sframe, user_id='user_id', item_id='business_id', target='stars', verbose=False)

Recsys training: model = popularity

Recsys training: model = item_similarity

Recsys training: model = factorization_recommender

Recsys training: model = ranking_factorization_recommender

And then evaluate them:

In [36]:
 for model in [pop_model, sim_model, fac_model, rank_fac_model]:
    name = model.__class__.__name__
    train_rmse = model.evaluate_rmse(train_sframe, target='stars')['rmse_overall']
    test_rmse = model.evaluate_rmse(test_sframe, target='stars')['rmse_overall']
    print '{} train RMSE: {}'.format(name, train_rmse)
    print '{} test RMSE: {}'.format(name, test_rmse)

PopularityRecommender train RMSE: 1.22558628247
PopularityRecommender test RMSE: 1.26124521478
ItemSimilarityRecommender train RMSE: 3.97754696406
ItemSimilarityRecommender test RMSE: 3.97934964672
FactorizationRecommender train RMSE: 0.386348157547
FactorizationRecommender test RMSE: 1.44804211375
RankingFactorizationRecommender train RMSE: 0.391552052851
RankingFactorizationRecommender test RMSE: 2.10284057685


Interestingly, the [Popularity Recommender](https://turi.com/products/create/docs/generated/graphlab.recommender.popularity_recommender.PopularityRecommender.html),
which scores items according to the number of times they're seen in the training set, actually does better than our simple averaging model. Because the library isn't open-source, and the documentation for it a little bit sparse, it's hard to figure out
exactly how it works, but the documentation does state that model that makes the same prediction for every user. To be fair, many Yelp users only review a single business, so perhaps such a model makes sense. In any case, this will definitely be a good baseline to compare against.

The matrix factorization-based recommenders (in particular the [Factorization Recommender](https://turi.com/products/create/docs/generated/graphlab.recommender.factorization_recommender.FactorizationRecommender.html))
appear to overfit the training set slightly, but there are many options for regularization and optimization parameters, so we can almost certainly improve them.

The [Item Similarity Recommender](https://turi.com/products/create/docs/generated/graphlab.recommender.item_similarity_recommender.ItemSimilarityRecommender.html)
is more along the lines of collaborative filtering, but because we aren't using any other attributes of businesses or users yet,
it's not a fair baseline (as evidenced by its terrible RMSE).

### Conclusions

We were able to train fairly strong baseline models of several kinds -- simple averaging, user-agnostic popularity, and matrix factorization -- all without considering any of the attributes of our data, apart from how highly who rated what. Hopefully we can achieve stronger results when we try collaborative filtering or combining any of the above methods with time-sensitive baselines as described in the Netflix result.